Title: julia computations benchmark

# [f(x,y) for x in, y in]

In [1]:
using BenchmarkTools

In [5]:
f!(x,y) = sin(x)+cos(y)
C = zeros(1000,1000)
@btime for y in 1:1000
       for x in 1:1000
       C[x,y] = f!(x,y)
       end
       end

  52.310 ms (1978000 allocations: 30.18 MiB)


for x, for y (Bad choice)

In [11]:
@btime for x in 1:1000
       for y in 1:1000
       C[x,y] = f!(x,y)
       end
       end

  52.861 ms (1978000 allocations: 30.18 MiB)


In [6]:
@btime Threads.@threads for y in 1:1000
       for x in 1:1000
       C[x,y] = f!(x,y)
       end
       end

  16.484 ms (1978021 allocations: 30.18 MiB)


Using multi threads and @inbounds

In [10]:
@btime Threads.@threads for y in 1:1000
       for x in 1:1000
       @inbounds C[x,y] = f!(x,y)
       end
       end

  14.812 ms (1978021 allocations: 30.18 MiB)


using generator

In [8]:
@btime [f!(x,y) for x in 1:1000, y in 1:1000];

  25.285 ms (2 allocations: 7.63 MiB)


In [14]:
using LoopVectorization
@btime @turbo for y in 1:1000
    for x in 1:1000
         C[x, y] = f!(x, y)
    end
end

  7.300 ms (11 allocations: 320 bytes)


In [19]:
function test!(C)
 @turbo for y in 1:1000
    for x in 1:1000
         C[x, y] = f!(x, y)
    end
end
end

@btime test!($C)

  7.423 ms (0 allocations: 0 bytes)


In [22]:
function test!(C)
  @turbo for y in 1:1000
    for x in 1:1000
         C[x, y] = f!(x, y)
    end
end
end

@btime test!($C)

  7.266 ms (0 allocations: 0 bytes)
